我们来构建一个非常经典、也非常直观的贝叶斯网络实例：草地湿润模型 (Wet Grass Model)。
这个问题完美地展示了贝叶斯网络如何进行因果推理和处理不确定性，特别是它著名的 **“解释相消 (Explaining Away)”** 效应。

一、 经典问题：我家的草地为什么湿了？

场景描述：
你早上醒来，发现自家院子里的草地是湿的。草地湿润（Wet Grass）可能有两个原因：
昨晚**下雨 (Rain)**了。
昨晚自动洒水器 (Sprinkler) 打开了。
同时，这两个原因又可能受到同一个上游因素的影响：
天气是否多云 (Cloudy)。
变量之间的因果关系（我们的常识/先验知识）：
多云会增加下雨的可能性。(Cloudy -> Rain)
多云会降低人们打开洒水器的可能性（因为觉得可能会下雨）。(Cloudy -> Sprinkler)
下雨会导致草地湿润。(Rain -> Wet Grass)
洒水器打开会导致草地湿润。(Sprinkler -> Wet Grass)
核心推理任务：
预测：如果我知道今天多云，那么草地湿润的概率有多大？
诊断：如果我观察到草地是湿的，那么原因是下雨的概率有多大？是洒水器的概率有多大？
高级推理：如果我观察到草地是湿的，并且我确认昨晚确实下雨了，那么洒水器也打开了的概率是变高了还是变低了？

二、 算法思想：构建因果图并量化信念

贝叶斯网络将这个问题分为两步解决：
定性建模：构建有向无环图 (DAG)
我们根据上面的因果关系，画出一个图。这个图的结构就是我们模型的骨架。
code
Code
Cloudy (C)
  /      \
 v        v
Sprinkler(S)  Rain(R)
 \      /
  v    v
 Wet Grass (W)
定量建模：定义条件概率表 (CPT)
我们为图中的每一个节点，根据其父节点，定义一个概率表。这部分是量化我们的信念。
(这里的概率是我们基于经验设定的)
P(Cloudy) - 根节点，没有父节点
P(C=True) = 0.5
P(Sprinkler | Cloudy)
P(S=T | C=T) = 0.1 (多云时，人们懒得开洒水器)
P(S=T | C=F) = 0.5 (晴天时，人们可能会开洒水器)
P(Rain | Cloudy)
P(R=T | C=T) = 0.8 (多云时，很可能下雨)
P(R=T | C=F) = 0.2 (晴天时，不太可能下雨)
P(Wet Grass | Sprinkler, Rain) - 有两个父节点，需要4个条目
P(W=T | S=T, R=T) = 0.99 (都开了，草地几乎肯定湿)
P(W=T | S=T, R=F) = 0.90 (只有洒水器，也湿)
P(W=T | S=F, R=T) = 0.80 (只有雨，也湿)
P(W=T | S=F, R=F) = 0.00 (都关了，草地是干的)
现在，我们的贝叶斯网络模型就完全定义好了。接下来，我们可以用它来进行推理了。

三、 Python 代码实现 (使用 pgmpy 库)

pgmpy 是Python中进行概率图模型建模和推理最专业的库。

In [2]:
import numpy as np
# 修正了这里的导入
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# 1. 定义贝叶斯网络结构
# 修正了这里的类名
model = DiscreteBayesianNetwork([('Cloudy', 'Sprinkler'), 
                                 ('Cloudy', 'Rain'),
                                 ('Sprinkler', 'WetGrass'),
                                 ('Rain', 'WetGrass')])

# 2. 定义条件概率分布 (CPDs)
# 这部分代码完全不需要改变

# P(Cloudy)
cpd_c = TabularCPD(variable='Cloudy', variable_card=2, values=[[0.5], [0.5]],
                   state_names={'Cloudy': ['False', 'True']})

# P(Sprinkler | Cloudy)
cpd_s = TabularCPD(variable='Sprinkler', variable_card=2, 
                   values=[[0.5, 0.9],  # P(S=F|C=F), P(S=F|C=T)
                           [0.5, 0.1]], # P(S=T|C=F), P(S=T|C=T)
                   evidence=['Cloudy'],
                   evidence_card=[2],
                   state_names={'Sprinkler': ['False', 'True'],
                                'Cloudy': ['False', 'True']})

# P(Rain | Cloudy)
cpd_r = TabularCPD(variable='Rain', variable_card=2,
                   values=[[0.8, 0.2],  # P(R=F|C=F), P(R=F|C=T)
                           [0.2, 0.8]], # P(R=T|C=F), P(R=T|C=T)
                   evidence=['Cloudy'],
                   evidence_card=[2],
                   state_names={'Rain': ['False', 'True'],
                                'Cloudy': ['False', 'True']})

# P(WetGrass | Sprinkler, Rain)
cpd_w = TabularCPD(variable='WetGrass', variable_card=2,
                   values=[[1.0, 0.1, 0.2, 0.01], # P(W=F | S,R)
                           [0.0, 0.9, 0.8, 0.99]],# P(W=T | S,R)
                   evidence=['Sprinkler', 'Rain'],
                   evidence_card=[2, 2],
                   state_names={'WetGrass': ['False', 'True'],
                                'Sprinkler': ['False', 'True'],
                                'Rain': ['False', 'True']})

# 3. 将CPDs添加到模型中
model.add_cpds(cpd_c, cpd_s, cpd_r, cpd_w)

# 4. 验证模型结构和CPDs是否正确
assert model.check_model()

# 5. 进行推理
# 这部分代码也完全不需要改变
inference = VariableElimination(model)

# --- 回答我们的核心推理任务 ---

print("--- 任务1: 预测 (正向推理) ---")
prob_wet_grass_given_cloudy = inference.query(variables=['WetGrass'], evidence={'Cloudy': 'True'})
print("如果多云，草地湿润的概率是:\n", prob_wet_grass_given_cloudy)

print("\n--- 任务2: 诊断 (反向推理) ---")
prob_rain_given_wet_grass = inference.query(variables=['Rain'], evidence={'WetGrass': 'True'})
print("如果草地湿润，原因是下雨的概率是:\n", prob_rain_given_wet_grass)

print("\n--- 任务3: 高级推理 ('解释相消'效应) ---")
prob_sprinkler_given_wet_grass = inference.query(variables=['Sprinkler'], evidence={'WetGrass': 'True'})
print("只知道草地湿润时，洒水器打开的概率是:\n", prob_sprinkler_given_wet_grass)

prob_sprinkler_given_wet_grass_and_rain = inference.query(variables=['Sprinkler'], 
                                                          evidence={'WetGrass': 'True', 'Rain': 'True'})
print("\n知道草地湿润且下雨时，洒水器打开的概率是:\n", prob_sprinkler_given_wet_grass_and_rain)

--- 任务1: 预测 (正向推理) ---
如果多云，草地湿润的概率是:
 +-----------------+-----------------+
| WetGrass        |   phi(WetGrass) |
+=================+=================+
| WetGrass(False) |          0.2568 |
+-----------------+-----------------+
| WetGrass(True)  |          0.7432 |
+-----------------+-----------------+

--- 任务2: 诊断 (反向推理) ---
如果草地湿润，原因是下雨的概率是:
 +-------------+-------------+
| Rain        |   phi(Rain) |
+=============+=============+
| Rain(False) |      0.2683 |
+-------------+-------------+
| Rain(True)  |      0.7317 |
+-------------+-------------+

--- 任务3: 高级推理 ('解释相消'效应) ---
只知道草地湿润时，洒水器打开的概率是:
 +------------------+------------------+
| Sprinkler        |   phi(Sprinkler) |
+==================+==================+
| Sprinkler(False) |           0.5894 |
+------------------+------------------+
| Sprinkler(True)  |           0.4106 |
+------------------+------------------+

知道草地湿润且下雨时，洒水器打开的概率是:
 +------------------+------------------+
| Sprinkler        |   phi(Sprinkler) |
+======